In [1]:
import cv2
import numpy as np
import pandas as pd
import time
import sys
import os

In [2]:
df = pd.read_csv('capstone_project_dataset.csv')

In [3]:
CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

net = cv2.dnn.readNet('/Users/abdullahalsheeha/Downloads/yolo-ds/yolov4_tiny_last.weights', '/Users/abdullahalsheeha/Downloads/yolo-ds/yolov4_tiny.cfg')

# classes = []## load the labels
# with open("/Users/abdullahalsheeha/Downloads/yolo-ds/obj.names","r") as f:
# 	classes=[line.strip() for line in f.readlines()]

# # generating colors for each object for later plotting
# layer_names=net.getLayerNames()
# output_layers=[layer_names[i[0]-1]for i in net.getUnconnectedOutLayers()]
##############################################
labels = open("/Users/abdullahalsheeha/Downloads/yolo-ds/obj.names").read().strip().split("\n")
# generating colors for each object for later plotting
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

# ### Make different color for each rectangle of a detected object
# colors=np.random.uniform(0,255,size=(len(classes),3))

In [4]:
print(labels)

['shawarma house', 'lusin', 'sultan steakhouse riyadh', 'sayadiyah express', 'cafe bateel', 'chicken republic', 'wakame', 'burj al hamam', 'al-saaj al-reefi', 'maestro pizza', 'khayal restaurant', 'sura korean fine dining', 'al baik', 'zafran indian bistro', 'myazu', 'taya restaurant', 'il vero', 'pattis france', 'steak house', 'hamra chicken', 'manoosha alreef', 'queen of india']


In [5]:
df['Resturant Name'] = df['Resturant Name'].str.lower()


In [6]:
for i in range(len(df)):
    df['Categories'][i] = df['Categories'][i].replace(',', '\n')
    df['Special Diets'][i] = df['Special Diets'][i].replace(',', '\n')

/Users/abdullahalsheeha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/abdullahalsheeha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
df

,Unnamed: 0,User ID,City,User Longitude,User Latitude,Resturant Name,Resturant Longitude,Resturant Latitude,Categories,Special Diets,Review Score,Comments
0,0,317,Jeddah,39.218564,21.736983,al-saaj al-reefi,39.137253,21.621797,Fast Food\n Middle Eastern\n Lebanese\n Medite...,Non,4.0,Wow... Loved this place
1,1,415,Riyadh,46.937884,24.551856,lusin,46.683451,24.697821,Mediterranean\n Middle Eastern\n Armenian,Vegetarian Friendly\n Vegan Options\n Gluten F...,4.6,Crust is not good.
2,2,425,Riyadh,46.501978,24.699156,la brasserie,46.684105,24.690431,French\n International,Vegetarian Friendly\n Vegan Options,2.7,Not tasty and the texture was just nasty.
3,3,456,Eastern Province,52.528577,26.575229,manoosha alreef,50.118295,26.445350,Lebanese\n Fast Food\n Mediterranean\n Middle ...,Vegetarian Friendly\n Vegan Options,4.7,Stopped by during the late May bank holiday of...
4,4,420,Eastern Province,53.446234,26.122919,queen of india,50.179829,26.328298,Indian,Vegetarian Friendly\n Vegan Options\n Gluten F...,4.9,The selection on the menu was great and so wer...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,166,Jeddah,39.304262,21.655744,aromi,39.109269,21.603596,Italian\n Mediterranean\n European,Vegetarian Friendly\n Vegan Options\n Gluten F...,2.8,I can't tell you how disappointed I was.
996,996,293,Eastern Province,51.736019,21.813979,tandoori house,50.129764,26.303790,Indian,Non,4.6,I think food should have flavor and texture an...
997,997,247,Eastern Province,55.466600,22.179759,al-sanbok restaurant,50.220734,26.271833,Seafood\n International\n Mediterranean,Vegetarian Friendly\n Vegan Options\n Gluten F...,4.2,Appetite instantly gone.
998,998,31,Riyadh,46.847037,24.553088,al orjouan,46.630201,24.665121,Mediterranean\n Middle Eastern\n International,Vegetarian Friendly\n Vegan Options\n Gluten F...,3.6,Overall I was not impressed and would not go b...


In [8]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
df['Resturant Name'].unique()

array(['al-saaj al-reefi', 'lusin', 'la brasserie', 'manoosha alreef',
       'queen of india', 'al bustan restaurant', 'zafran indian bistro',
       'maestro pizza', 'khayal restaurant', 'sura korean fine dining',
       'byblos restaurant', 'il vero', 'pattis france', 'lotus',
       'shawarma house', 'hamra chicken', 'aromi', 'tokyo',
       'sayadiyah express', 'tandoori house', 'taya restaurant',
       'the blue restaurant', 'burj al hamam', 'wakame', 'al baik',
       'al orjouan', 'sultan steakhouse riyadh', 'steak house',
       'cafe bateel', 'ginger leaf', 'abu nawas', 'chicken republic',
       'myazu', 'al-sanbok restaurant'], dtype=object)

In [10]:
df2 = {}

for i in labels:
    #print([i, round(df[df['Resturant Name'] == i]['Review Score'].mean(), 2), df[df['Resturant Name'] == i].iloc[0]['Categories'], df[df['Resturant Name'] == i].iloc[0]['Special Diets']])
    df2[i] = [round(df[df['Resturant Name'] == i]['Review Score'].mean(), 1), df[df['Resturant Name'] == i].iloc[0]['Categories'], df[df['Resturant Name'] == i].iloc[0]['Special Diets']]

In [11]:
path_name = "obj 2/2.jpg"
image = cv2.imread(path_name)
file_name = os.path.basename(path_name)
filename, ext = file_name.split(".")

In [12]:
h, w = image.shape[:2]
# create 4D blob
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

In [13]:
# sets the blob as the input of the network
net.setInput(blob)
# get all the layer names
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# feed forward (inference) and get the network output
# measure how much it took in seconds
start = time.perf_counter()
layer_outputs = net.forward(ln)
time_took = time.perf_counter() - start
print(f"Time took: {time_took:.2f}s")

Time took: 0.06s


In [14]:
font_scale = 1
thickness = 1
boxes, confidences, class_ids = [], [], []
# loop over each of the layer outputs
for output in layer_outputs:
    # loop over each of the object detections
    for detection in output:
        # extract the class id (label) and confidence (as a probability) of
        # the current object detection
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        # discard out weak predictions by ensuring the detected
        # probability is greater than the minimum probability
        if confidence > CONFIDENCE:
            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the bounding
            # box followed by the boxes' width and height
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [15]:
idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)

In [16]:
# ensure at least one detection exists
if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
        # extract the bounding box coordinates
        x, y = boxes[i][0], boxes[i][1]
        w, h = boxes[i][2], boxes[i][3]
        # draw a bounding box rectangle and label on the image
        color = [int(c) for c in colors[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
        text = f"{labels[class_ids[i]]}" 
        text1 = f"rating: {df2[labels[class_ids[i]]][0]}"
        text2 = f"Categories: {df2[labels[class_ids[i]]][1]}"
        text3 = f"Special Diets: {df2[labels[class_ids[i]]][2]}"
        print(df2[labels[class_ids[i]]])
        # calculate text width & height to draw the transparent boxes as background of the text
        (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
        text_offset_x = x -50
        text_offset_y = y + 40
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 250, text_offset_y + text_height+100))
        overlay = image.copy()
        cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
       
        
        # add opacity (transparency to the box)
        image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
        # now put the text (label: confidence %)
        cv2.putText(image, text, (x-50, y + 60), cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.7, color=(0, 0, 0), thickness=thickness, lineType = cv2.LINE_AA)
        cv2.putText(image, text1, (x-50 , y+80), cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.35, color=(0, 0, 0), thickness=thickness, lineType = cv2.LINE_AA)
        y2, dy = y+93, 12
        for i, line1 in enumerate(text3.split('\n')):
            y3 = y2 + i*dy    
            cv2.putText(image, line1, (x-50 , y3), cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.35, color=(0, 0, 0), thickness=thickness, lineType = cv2.LINE_AA)
        y0, dy = y+130, 12
        for i, line in enumerate(text2.split('\n')):
            y1 = y0 + i*dy
            #cv2.putText(image, line, (x-120 , y+80), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            cv2.putText(image, line, (x-50, y1), cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.35, color=(0, 0, 0), thickness=thickness, lineType = cv2.LINE_AA)
        
        

[4.1, 'Chinese\n Indian\n Seafood\n Grill\n Arabic\n Barbecue', 'Non']


In [17]:
# cv2.imshow('x',image)
cv2.imwrite(filename + "_yolo4." + ext, image)

True